# Maxwell unit: compliance function & evolution of deformation 

***

<center><i>Petr Havlásek (c) 2023-24, petr.havlasek@cvut.cz</i></center>

***

In [1]:
online = False

if (online):
    import micropip
    await micropip.install('ipywidgets')

import math
import numpy as np    
    
import matplotlib.pyplot as plt
    
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

from IPython.display import display


## Compliance function of a Maxwell unit
**Notation:**

>`E` ... spring stiffness [MPa]<br>

>`eta` ... dashpot viscosity [MPa day]<br>

>`t` ... time of interest [day]<br>

>`tt` ... time of loading [day]<br>

In [2]:
def Maxwell_J_func(E, eta, t, tt):

    if t >= tt:
        dt = t-tt

        return 1./ E  + dt / eta
            
    else:
        return 0.

## Parameters definition

In [3]:
# spring stiffness [MPa]
E_ref = 1.
# dashpot viscosity [MPa * day]
eta_ref = 1.
# reference relaxation time
tau_ref = eta_ref / E_ref

# variables ranges: powers of 10
# stiffness
log_E_min = -3
log_E_max = 3

# viscosity
log_eta_min = -3
log_eta_max = 3

log_tau_min = -1
log_tau_max = 1

# number of time steps
t_div = 100

## Compliance function

In [4]:

def plot_Maxwell_J_func(E, eta, log_scale = True, show_tau = False, show_reference = True):

    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )
        
    ax.clear()

    # reference data with E_ref and eta_ref
    if show_reference:
        J_ref = []
        for t in times:
            J_ref.append( Maxwell_J_func(E_ref, eta_ref, t, 0.) )

        ax.plot(times, J_ref, lw=1.5, color="red", linestyle='dashed', label='Jref(t)')
        
        if (show_tau):
            ax.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    J = []
    for t in times:
        J.append( Maxwell_J_func(E, eta, t, 0.) )
    ax.plot(times, J, lw=2., color="blue", label=r'J(t)')
    if (show_tau):
        ax.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax.set_xscale('log')
        ax.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax.set_xlim([0., 10**log_tau_max])

    ax.grid(True)
    ax.legend()
    ax.set_xlabel('Duration of loading, t-t\' [day]')
    ax.set_ylabel('Compliance, J [1/MPa]')  
    
    display(fig)

### GUI for compliance function

In [5]:


output = widgets.Output()

fig, ax = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig)


E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')

aux = interactive(plot_Maxwell_J_func, E = E_slide, eta = eta_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)


interactive(children=(FloatLogSlider(value=1.0, description='E [MPa]', max=3.0, min=-3.0, step=0.25), FloatLog…

## Strain evolution under constant stress level

In [10]:
def plot_Maxwell_deformation(E, eta, sigma, log_scale = True, show_tau = False, show_reference = True):

    if (log_scale):
        times = np.logspace(log_tau_min, log_tau_max, num = t_div )
    else:
        times = np.linspace(0., 10**log_tau_max, num = t_div )

    ax2.clear()        

    # reference data with E_ref and eta_ref
    if show_reference:
        eps_e_ref = []
        eps_v_ref = []
        for t in times:
            eps_e_ref.append( sigma / E_ref  )
            eps_v_ref.append( sigma * t / eta_ref )

        ax2.plot(times, eps_e_ref, lw=1.5, color="red", linestyle='dashed', label='eps_e_ref(t)')
        ax2.plot(times, eps_v_ref, lw=1.5, color="red", linestyle='solid', label='eps_v_ref(t)')
        
        if (show_tau):
            ax2.axvline(x = tau_ref, lw=1.5, color="red", linestyle = "--")

    # data for user-defined parameters
    eps_e = []
    eps_v = []
    for t in times:
        eps_e.append( sigma / E  )
        eps_v.append( sigma * t / eta )

    ax2.plot(times, eps_e, lw=2.0, color="blue", linestyle='dashed', label='eps_e(t)')
    ax2.plot(times, eps_v, lw=2.0, color="blue", linestyle='solid', label='eps_v(t)')
    
    if (show_tau):
        ax2.axvline(x = eta/E, lw=1.5, color="blue", linestyle = "--")
    
    if (log_scale):
        ax2.set_xscale('log')
        ax2.set_xlim([10**log_tau_min, 10**log_tau_max])
    else:
        ax2.set_xlim([0., 10**log_tau_max])


    ax2.grid(True)
    ax2.legend()
    ax2.set_xlabel('Duration of loading, t-t\' [day]')
    ax2.set_ylabel('Deformation, epsilon [$10^{-6}$]')
    
    display(fig2)

In [11]:
## GUI for strain evolution

In [12]:
output = widgets.Output()

fig2, ax2 = plt.subplots(1, 1, figsize=(10,5))
plt.rcParams.update({'font.size': 14})
plt.close(fig2)

## GUI
E_slide = widgets.FloatLogSlider(min=log_E_min, max=log_E_max, value=1., step=0.25, description='E [MPa]')
eta_slide = widgets.FloatLogSlider(min=log_eta_min, max=log_eta_max, value=1., step=0.25, description='eta [MPa day]')
sigma_slide = widgets.FloatSlider(min=0., max=100., value=10., step=1., description='sigma [MPa]')

aux = interactive(plot_Maxwell_deformation, E = E_slide, eta = eta_slide, sigma = sigma_slide, log_scale = False, show_tau = False, show_reference = True)

display(aux)


interactive(children=(FloatLogSlider(value=1.0, description='E [MPa]', max=3.0, min=-3.0, step=0.25), FloatLog…